In [ ]:
import seaborn  
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statistics
from sklearn.linear_model
import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')
%pip install lxml
%pip install html5lib


# Analysis of Povety in America and the Implications of Race on Proverty Status

## Data Collection, Processing, and Cleaning

### Table 1: CPS Population and Per Capita Money Income, All People: 1967 to 2021

In [99]:
data1 = pd.read_csv("p01ar.csv", header=6) #header starts in row 6
data1.head(7)

,Year,Number (thousands),Per capita income,Unnamed: 3
0,NaN,NaN,Current\ndollars,2021\ndollars
1,2021,"328,722","41,285","41,285"
2,2020 (41),"328,059","38,931","40,749"
3,2019,"325,268","39,156","41,496"
4,2018,"324,356","36,080","38,929"
5,2017 (40),"323,144","35,048","38,737"
6,2017,"323,156","34,489","38,119"


In [100]:
data1 = data1.rename(columns = {"Per capita income": "Current PCI in Dollars", "Unnamed: 3":"2021 PCI in Dollars"}) #rename headers
data1 = data1.drop(index = [0]) # now can drop the first row 
data1.head(7)
#PCI = Per Capita Income

,Year,Number (thousands),Current PCI in Dollars,2021 PCI in Dollars
1,2021,"328,722","41,285","41,285"
2,2020 (41),"328,059","38,931","40,749"
3,2019,"325,268","39,156","41,496"
4,2018,"324,356","36,080","38,929"
5,2017 (40),"323,144","35,048","38,737"
6,2017,"323,156","34,489","38,119"
7,2016,"320,372","33,205","37,490"


In [117]:
data1['Year'] = data1['Year'].apply(lambda item : (str(item[:4])) if str.__contains__(str(item),'(') == True else item) #want to remove the numbers in paranthesis next to the years
data1['Number (thousands)'] = data1['Number (thousands)'].apply(lambda item : (item.replace(",", "") if str.__contains__(str(item),',') == True else item))
data1['Current PCI in Dollars'] = data1['Current PCI in Dollars'].apply(lambda item : (item.replace(",", "") if str.__contains__(str(item),',') == True else item))
data1['2021 PCI in Dollars'] = data1['2021 PCI in Dollars'].apply(lambda item : (item.replace(",", "") if str.__contains__(str(item),',') == True else item))
data1 = data1.astype({'Number (thousands)': int, 'Current PCI in Dollars': int, '2021 PCI in Dollars':int})
data1.head(15)
#changed last 3 columns into integers in case calculation is necessary later

,Year,Number (thousands),Current PCI in Dollars,2021 PCI in Dollars
1,2021,328722,41285,41285
2,2020,328059,38931,40749
3,2019,325268,39156,41496
4,2018,324356,36080,38929
5,2017,323144,35048,38737
6,2017,323156,34489,38119
7,2016,320372,33205,37490
8,2015,318868,31653,36198
9,2014,316168,30176,34569
10,2013,313443,30027,34980


### Table 2: People in Poverty by Nativity: 1993 to 2021

In [118]:
data2 = pd.read_excel("hstpov16.xlsx", header = 3) #note: 2 levels of headers
data2.head()

,Year,All people,Unnamed: 2,Unnamed: 3,Native,Unnamed: 5,Unnamed: 6,Foreign-born,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,NaN,NaN,Naturalized citizen,NaN,NaN,Not a citizen,NaN,NaN
1,NaN,Total,Number in poverty,Percent in poverty,Total,Number in poverty,Percent in poverty,Total,Number in poverty,Percent in poverty,Total,Number in poverty,Percent in poverty,Total,Number in poverty,Percent in poverty
2,2021,328191,37933,11.6,281383,31083,11,46808,6850,14.6,22874,2428,10.6,23934,4422,18.5
3,2020 (1),327570,37548,11.5,282408,31485,11.1,45162,6063,13.4,22800,2087,9.2,22362,3977,17.8
4,2019,324754,33984,10.5,279867,28342,10.1,44886,5643,12.6,22746,2038,9,22140,3605,16.3


In [121]:
#just want Native groups
dataNew = data2.loc[:, ["Year","Native","Unnamed: 5", "Unnamed: 6"]]
dataNew['Year'] = dataNew['Year'].apply(lambda item : (str(item[:4])) if str.__contains__(str(item),'(') == True else item) #want to remove the numbers in paranthesis next to the years
dataNew = dataNew.rename(columns = {"Native": "Total", "Unnamed: 5":"Number in Poverty", "Unnamed: 6": "Percent in Poverty"}) #rename headers
dataNew = dataNew.drop(index = [0,1])
dataNew.head(7)

,Year,Total,Number in Poverty,Percent in Poverty
2,2021,281383,31083,11
3,2020,282408,31485,11.1
4,2019,279867,28342,10.1
5,2018,278051,31828,11.4
6,2017,277131,33143,12
7,2017,277158,33095,11.9
8,2016,276089,33999,12.3


### Table 3: Poverty Status of People by Family Relationship, Race, and Hispanic Origin: 1959 to 2021

In [97]:
data3 = pd.read_excel("hstpov2.xlsx", header = 4 ) #several headers, first one starts on line 4
data3.head()
# dont any drop the rows or the rest of the dataframes will be affected too
# clean up data3 here so the subsets of data retrieved from it are cleaner already 

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,All families,NaN,NaN,"Families with female householder, no spouse pr...",NaN,NaN,NaN,NaN,NaN
1,NaN,Total,Below poverty,NaN,Total,Below poverty,NaN,Total,Below poverty,NaN,Total,Below poverty,NaN
2,NaN,NaN,Number,Percent,NaN,Number,Percent,NaN,Number,Percent,NaN,Number,Percent
3,2021,328191,37933,11.6,263764,25077,9.5,48476,12264,25.3,63398,12513,19.7
4,2020 (1),327570,37548,11.5,264437,25232,9.5,48984,12469,25.5,62097,11963,19.3


In [98]:
#plitting up data
#add the other years
dataAllRaces= data3.iloc[4:68,:]
dataAllRaces.head()

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
4,2020 (1),327570,37548,11.5,264437,25232,9.5,48984,12469,25.5,62097,11963,19.3
5,2019,324754,33984,10.5,263696,22431,8.5,46255,11262,24.3,60117,11300,18.8
6,2018,323847,38146,11.8,262010,25489,9.7,46660,12491,26.8,60768,12287,20.2
7,2017 (2),322548,39564,12.3,261599,26720,10.2,47517,13525,28.5,59835,12465,20.8
8,2017,322549,39698,12.3,260709,26766,10.3,47999,13378,27.9,60786,12593,20.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,1963,187258,36436,19.5,176076,31498,17.9,N,7646,47.7,11182,4938,44.2
64,1962,184276,38625,21,173263,33623,19.4,N,7781,50.3,11013,5002,45.4
65,1961,181277,39628,21.9,170131,34509,20.3,N,7252,48.1,11146,5119,45.9
66,1960,179503,39851,22.2,168615,34925,20.7,N,7247,48.9,10888,4926,45.2


In [28]:
dataWhite = data3.iloc[71:143,:]
dataWhite.head()

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
71,NaN,Total,Below poverty,NaN,Total,Below poverty,NaN,Total,Below poverty,NaN,Total,Below poverty,NaN
72,NaN,NaN,Number,Percent,NaN,Number,Percent,NaN,Number,Percent,NaN,Number,Percent
73,2021,248909,24919,10,199319,15795,7.9,28961,6575,22.7,48788,8877,18.2
74,2020 (1),249083,25183,10.1,200217,16456,8.2,29829,6993,23.4,48044,8456,17.6
75,2019,248086,22512,9.1,200954,14295,7.1,27848,6007,21.6,46332,7998,17.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1962,162842,26672,16.4,153348,22613,14.7,N,4089,37.9,9494,4059,42.7
140,1961,160306,27890,17.4,150717,23747,15.8,N,4062,37.6,9589,4143,43.2
141,1960,158863,28309,17.8,149458,24262,16.2,N,4296,39,9405,4047,43
142,1959,156956,28484,18.1,147802,24443,16.5,N,4232,40.2,9154,4041,44.1


In [33]:
dataBlack = data3.iloc[236:300,:]
dataBlack.head()

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
236,2021,43976,8583,19.5,34323,6202,18.1,14035,4328,30.8,9514,2315,24.3
237,2020 (1),43694,8556,19.6,34451,6062,17.6,14082,4214,29.9,9141,2434,26.6
238,2019,42965,8073,18.8,34033,5777,17,13939,4118,29.5,8863,2271,25.6
239,2018,42773,8884,20.8,33237,6242,18.8,13500,4277,31.7,9388,2584,27.5
240,2017 (2),42477,9224,21.7,33261,6594,19.8,13986,4811,34.4,9064,2573,28.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1969,22011,7095,32.2,20192,6245,30.9,5537,3225,58.2,1819,850,46.7
296,1968,21944,7616,34.7,N,6839,33.7,N,3312,58.9,N,777,46.3
297,1967 (22),21590,8486,39.3,N,7677,38.4,N,3362,61.6,N,809,49.3
298,1966,21206,8867,41.8,N,8090,40.9,N,3160,65.3,N,777,54.4


In [38]:
dataAsian= data3.iloc[332:354,:]
dataAsian.head()

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
332,2021,20676,1922,9.3,17674,1206,6.8,1989,301,15.1,2972,714,24
333,2020 (1),20352,1645,8.1,17550,1080,6.2,1783,310,17.4,2754,553,20.1
334,2019,19926,1464,7.3,17134,946,5.5,1576,254,16.1,2752,518,18.8
335,2018,19768,1996,10.1,16765,1243,7.4,1686,327,19.4,2946,732,24.8
336,2017 (2),19526,1891,9.7,16748,1220,7.3,1715,288,16.8,2737,652,23.8
337,2017,19475,1953,10,16666,1276,7.7,1757,275,15.7,2758,674,24.4
338,2016,18879,1908,10.1,16220,1179,7.3,1657,326,19.7,2627,715,27.2
339,2015,18241,2078,11.4,15597,1260,8.1,1435,222,15.5,2556,784,30.7
340,2014,17790,2137,12,15261,1391,9.1,1725,315,18.2,2431,713,29.3
341,2013 (3),17257,2255,13.1,15057,1589,10.6,1574,442,28.1,2180,661,30.3


In [43]:
dataNA= data3.iloc[406:448,:] 
dataNA.head()

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
406,2021,4109,998,24.3,3432,759,22.1,999,434,43.5,649,218,33.5
407,2020 (1),3950,791,20,3271,591,18.1,877,303,34.6,665,194,29.2
408,2019,3735,677,18.1,3076,481,15.6,701,231,33,652,191,29.3
409,2018,3663,808,22.1,2993,597,20,828,233,28.1,656,209,31.8
410,2017 (2),3616,790,21.8,2973,573,19.3,888,299,33.7,618,206,33.4
411,2017,3619,750,20.7,2971,541,18.2,884,299,33.8,629,198,31.5
412,2016,3701,811,21.9,3037,598,19.7,892,323,36.2,635,189,29.7
413,2015,4269,995,23.3,3534,742,21,1090,481,44.1,701,231,32.9
414,2014,3808,1013,26.6,3225,791,24.5,940,445,47.4,559,210,37.6
415,2013 (3),3074,837,27.2,2587,664,25.7,840,359,42.7,485,173,35.6


In [47]:
dataHispanic = data3.iloc[480:531,:] 
dataHispanic.head()

,Year,All people,Unnamed: 2,Unnamed: 3,People in families,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unrelated individuals,Unnamed: 11,Unnamed: 12
480,2021,62482,10690,17.1,53549,8288,15.5,12382,3754,30.3,8643,2303,26.6
481,2020 (1),61769,10520,17,53621,8379,15.6,12856,3866,30.1,7871,1992,25.3
482,2019,60602,9545,15.7,52743,7587,14.4,12248,3512,28.7,7627,1878,24.6
483,2018,59957,10526,17.6,52041,8368,16.1,11939,3716,31.1,7645,2047,26.8
484,2017 (2),59051,10816,18.3,51651,8760,17,12155,4274,35.2,7063,1946,27.6
485,2017,59053,10790,18.3,51517,8708,16.9,12244,4198,34.3,7206,1954,27.1
486,2016,57556,11137,19.4,50525,9200,18.2,11926,4136,34.7,6697,1793,26.8
487,2015,56780,12133,21.4,49524,10109,20.4,11878,4401,37.1,6884,1876,27.2
488,2014,55504,13104,23.6,48296,10853,22.5,11919,4817,40.4,6776,1981,29.2
489,2013 (3),54181,13356,24.7,47266,11128,23.5,13060,5406,41.4,6414,1915,29.9


In [ ]:
# header = {
#     'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
#     'From': 'pleaseletmein@gmail.com'}
# povByRace = requests.get('https://www.kff.org/other/state-indicator/poverty-rate-by-raceethnicity/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D', 
#                                  headers = header)
# soup1 = BeautifulSoup(povByRace.content, 'html.parser')
# print(soup1.prettify())

# table1 = soup1.findAll('table')

# tables = pd.read_html(str(table1))
# df = tables[0]

# df


povByRace = pd.read_csv("povRace.csv", header=2)
povByRace.head(53)